# Notes on Conversion of pwmig to parallel_pwmig

March 12, 2021

This notebook is my working notes describing how I converted the original pwmig package to the revised version
I'm (tentatively) calling parallel_pwmig.  This is an extensive conversion because the revision will use the 
MsPASS framework.  There is a related directory in docs/design that contains preliminary thoughts on how this 
conversion be done. 

## Initial repository setup
The first thing I needed to do was assemble components from other directories.  Because pwmig originally had a dependency on antelope that involved moving some code from antelope contrib.   I also copied files from the original pwmig repository.  The files are a complete reorganization in directories from the parent code but there were no name changes.  Some files were not copied that were already in mspass or I knew were unnecessary.
I checked these into the repository master branch to preserve the entire history chain of the code.  

I then used the mspass cmake configuration files in include and source directories to set up the 
cmake system.  To get started I just built the root and linking Cmake file that define how cmake 
works through the directory chain.  The reason is that right now I'm no positive what will be converted
to python and what will be left as C++ main programs.   Also the correct library structure isn't clear 
either.  Hence, I'll kick that can down the road.  I'll concentrate instead on some of the initial code hacking.
Not sure how much of that I will actually record here.   It is the usual think of symbol changes, changes 
to include filel names, and configuration errors.   Warning is what is recorded below will likely be 
incomplete, but will try to record high points.  I'll make notes on each source directory.   

**Hypocenter**.   Quickly discovered I missed copying coords and stock libraries I had adapted for pwmig from 
the public domain version of antelope (Datascope).   Copied that to utility areas.  Removed the elog.h and 
elog.c file that we don't want her.  elog calls are going to be generic problem I will need to fix.  The seispp version of hypocenter had travel time methods embedded in the class.  That was a dumb design 
that I removed in the conversion.  I did, however, retain convenience methods for computing distance, 
seaz, and esaz.   NOTE:  this class might end up moving into MsPASS.

A good thing came from this in converting this function.  I used the keywords.h file to remove the literal 
key names used previously to fetch lat,lon, depth, and origin time in Metadata driven constructors.

**EventCatalog**.   The main reason I needed to convert Hypocenter was to use this C++ class that 
was used in telecluster to subset events into radial grid segments.   The implementation used does 
not translate easily at all to python and would be compute intensive if done with a dumber algorithm.
The original EventCatalog used Antelopes database to read event data.  It had two constructors 
using an antelope database handle.   For this application using pymongo I am removing those constructors
and depending on the use of append methods.  This will drastically slow construction as it will require 
a python load but this should not be an issue for telecluster as the number of teleseisms of use for 
this kind of processing is currently not that enormous relative to current computer memory sizes.  Certainly not something to worry about until it proves to be a problem. 

Note did a lot of interface changes to make proper use of const. 

**gclgrid**  This is a much bigger job due to the shear size of this library.   I debated a big reorganization and/or cleaning up problems with the design (this was my first major c++ library) but decided that was counterproductiv.  

For the rest of this I will write down the key stages in the modifications.

1.  The include changes needed in all of these files.
2.  Tedious job of using const appropriately in all the classes.  At the same time I cleared all the sections 
    within the ifndefs for use with antelope.  This created long list of compilation issues dealing with 
    const mismatches.   The dark side of fixing that kind of thing.
3.  Working through this made me realize an obvious gap of a program that needed conversion:  makegclgrid. 
    Added a new design page for that program.
4.  Another thing I cleared was GCLgrid_error was depricated for GCLgridError that was set up as a child of 
    std::exception
5.  Added a constructor for all classes with one arg; his generic signature:  GCLgridtype(const Metadata&)
    All depend on having valid dir, dfile, and foff entries in the metadata container.
6.  Decided to use a namespace pwmig::gclgrid.   That created another lengthy set of compilation errors 
    that had to be handled.  Note this and the const changes were both not essential for a conversion to 
    mspass but something I elected to do since I was going through the whole code base anyway.

## Build System

March 15-16, 2021

After initial changes to pwmig for the mspass framework I realized I needed a mechanism to more effectivly use the build system with cmake.  mspass installs compiled shared libraries and installs them and include files in a area defined by MSPASSHOME.  Seeking help from Ian on how to do his more automatically.   The fallback is to just require the MSPASSHOME environment variable.   Will update this whe I get an answer.

There may be a way to automatically do a cmake package someday but for now am going the environmeent variable route.  To do that I did the following:
1.  Had to build the full C libraries in mspass with the :code:`make_install` method.  That required setting an 
    environment variable MSPASS_HOME to a writeable directory.  
2.  In the top level CMakeList.txt file I added a line to extract MSPASS_HOME from the environment and set 
    a slight variant as the cmake variable MSPASSHOME.   All directories that need mspass include files need
    a ${MSPASSHOME} inserted appropriately (gclgrid was the first example I found that worked.)   